In [1]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import word_tokenize
import pandas as pd
import string
import stanza
import nltk
import fs
import ast

In [2]:
# Download files
stanza.download("es") 
nltk.download('stopwords')
nltk.download('wordnet')

2024-11-24 15:35:36 INFO: Downloaded file to /root/stanza_resources/resources.json
2024-11-24 15:35:36 INFO: Downloading default packages for language: es (Spanish) ...


2024-11-24 15:35:40 INFO: Downloaded file to /root/stanza_resources/es/default.zip
2024-11-24 15:35:44 INFO: Finished downloading models and saved to /root/stanza_resources
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
INTERIM_FILES_DIR = fs.open_fs("MCD-NLP-HPT/data/interim")
INTERIM_COMMENTS_ITEMS_DIR = INTERIM_FILES_DIR.getsyspath("Items.csv")

In [3]:
INTERIM_COMMENTS_CLEANED_DIR = INTERIM_FILES_DIR.getsyspath("Comments_cleaned.csv")

In [4]:
df_items = pd.read_csv(INTERIM_COMMENTS_ITEMS_DIR)
df_items

,kind,etag,id,snippet.channelId,snippet.videoId,snippet.topLevelComment.kind,snippet.topLevelComment.etag,snippet.topLevelComment.id,snippet.topLevelComment.snippet.channelId,snippet.topLevelComment.snippet.videoId,...,snippet.topLevelComment.snippet.viewerRating,snippet.topLevelComment.snippet.likeCount,snippet.topLevelComment.snippet.publishedAt,snippet.topLevelComment.snippet.updatedAt,snippet.canReply,snippet.totalReplyCount,snippet.isPublic,replies.comments,category_id,category_description
0,youtube#commentThread,35SzVbfAjbh-WxxI6CDcz8jJalE,Ugz0QmXyVLSM0nUK7kB4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,cQ9RLDhq6JY,youtube#comment,yopv5G2xkuk2ZN4mdQMDcEtKmS4,Ugz0QmXyVLSM0nUK7kB4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,cQ9RLDhq6JY,...,none,0,2024-07-15T12:41:43Z,2024-07-15T12:41:43Z,True,0,True,NaN,1,Quejas o sugerencias de mejora
1,youtube#commentThread,D5l-LBB9fD-VlBImz-EN9K8mTmM,UgyJhRBhVcbyStZdxjR4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,cQ9RLDhq6JY,youtube#comment,avm1UM6tZkxbnMkp59t7_36GZrk,UgyJhRBhVcbyStZdxjR4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,cQ9RLDhq6JY,...,none,0,2024-05-26T21:41:04Z,2024-05-26T21:41:04Z,True,0,True,NaN,7,Comentarios generales
2,youtube#commentThread,iKN6FQSPLo3xUlizd--6R0h2SN8,UgzT1q2am7kMEM5ckDp4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,cQ9RLDhq6JY,youtube#comment,g6vw71KL8rJyQPLYPCUCm4O6rbA,UgzT1q2am7kMEM5ckDp4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,cQ9RLDhq6JY,...,none,0,2024-05-23T22:06:21Z,2024-05-23T22:06:21Z,True,0,True,NaN,7,Comentarios generales
3,youtube#commentThread,hkErqy7RZOwAKS6YyNXqwPFTCAg,Ugyckucg38lqZFdZhaR4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,cQ9RLDhq6JY,youtube#comment,soQFV-F5m7bRA03fMjI1IoKQrGs,Ugyckucg38lqZFdZhaR4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,cQ9RLDhq6JY,...,none,0,2024-05-05T09:24:57Z,2024-05-05T09:24:57Z,True,0,True,NaN,6,Comentarios humorísticos o memes
4,youtube#commentThread,9c50dNDxQ96ZYyFK1aeJYj4ofnQ,UgzhZHVt-tEbDpbCvtB4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,cQ9RLDhq6JY,youtube#comment,wLebngIiKaOrIw1ibNjosSTEyFc,UgzhZHVt-tEbDpbCvtB4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,cQ9RLDhq6JY,...,none,0,2024-04-21T21:09:08Z,2024-04-21T21:09:08Z,True,0,True,NaN,5,Felicitaciones y agradecimientos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,youtube#commentThread,8wSxk2QdKgz2v0AUy3EsWvUOxSU,Ugyuq-tSoKFj7HEnAQ54AaABAg,UCeYSnd34IPe57sM8hkjWhrw,t3MogAmkMUg,youtube#comment,G_pL0x_Nmvi7C0kj8fHi_IWkcCA,Ugyuq-tSoKFj7HEnAQ54AaABAg,UCeYSnd34IPe57sM8hkjWhrw,t3MogAmkMUg,...,none,0,2024-08-19T19:46:46Z,2024-08-19T19:46:46Z,True,0,True,NaN,6,Comentarios humorísticos o memes
1022,youtube#commentThread,X4xqkZX0x2OmHUMWh4fe2ck4iGA,Ugy2h_vD89gf_8eeFph4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,t3MogAmkMUg,youtube#comment,fWBX1bGUjpM7nRaQcNKN1P7BpFQ,Ugy2h_vD89gf_8eeFph4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,t3MogAmkMUg,...,none,0,2024-08-19T19:33:41Z,2024-08-19T19:33:41Z,True,0,True,NaN,2,Propuestas y recomendaciones
1023,youtube#commentThread,btoZQh1KTx4_A0e8kLI3d4BAUmg,Ugx7vBivUc-z-n8H76t4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,t3MogAmkMUg,youtube#comment,CcywvY3FEol-BcpzJpjundijK6k,Ugx7vBivUc-z-n8H76t4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,t3MogAmkMUg,...,none,1,2024-08-19T19:33:18Z,2024-08-19T19:33:18Z,True,0,True,NaN,1,Quejas o sugerencias de mejora
1024,youtube#commentThread,Xf-HPCYvwS0mb7JyqR4t_eoHZx4,Ugzvu7xFsIUo8ibNXPJ4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,t3MogAmkMUg,youtube#comment,FLjbXmI4uIIbM1ZQNW-hMSKWARM,Ugzvu7xFsIUo8ibNXPJ4AaABAg,UCeYSnd34IPe57sM8hkjWhrw,t3MogAmkMUg,...,none,0,2024-08-19T19:13:00Z,2024-08-19T19:13:00Z,True,0,True,NaN,4,Correcciones o datos adicionales


In [6]:
df_cleaned = df_items[[
    'id', 
    'snippet.videoId',
    'snippet.topLevelComment.snippet.textOriginal',
    'snippet.topLevelComment.snippet.authorDisplayName',
    'snippet.topLevelComment.snippet.likeCount',
    'snippet.topLevelComment.snippet.publishedAt',
    'category_id',
    'category_description'
]].copy()
df_cleaned.columns = [
    'id', 
    'videoId',
    'textOriginal',
    'authorDisplayName',
    'likeCount',
    'publishedAt',
    'category_id',
    'category_description'
]
df_cleaned

,id,videoId,textOriginal,authorDisplayName,likeCount,publishedAt,category_id,category_description
0,Ugz0QmXyVLSM0nUK7kB4AaABAg,cQ9RLDhq6JY,"Me gusta su podcast, pero solo he escuchado lo...",@fabianbecerra2272,0,2024-07-15T12:41:43Z,1,Quejas o sugerencias de mejora
1,UgyJhRBhVcbyStZdxjR4AaABAg,cQ9RLDhq6JY,weeeeey,@SantiagoItzcoatl,0,2024-05-26T21:41:04Z,7,Comentarios generales
2,UgzT1q2am7kMEM5ckDp4AaABAg,cQ9RLDhq6JY,Les voy a enviar una foto de TARAPACA,@Sonyeke_1,0,2024-05-23T22:06:21Z,7,Comentarios generales
3,Ugyckucg38lqZFdZhaR4AaABAg,cQ9RLDhq6JY,Entonces Teca esta solterooooooooooooooo! ❤ ...,@alejandranavarro464,0,2024-05-05T09:24:57Z,6,Comentarios humorísticos o memes
4,UgzhZHVt-tEbDpbCvtB4AaABAg,cQ9RLDhq6JY,"Me encantan sus podcast, los escucho en spotif...",@fiorellamartinez873,0,2024-04-21T21:09:08Z,5,Felicitaciones y agradecimientos
...,...,...,...,...,...,...,...,...
1021,Ugyuq-tSoKFj7HEnAQ54AaABAg,t3MogAmkMUg,Justo a tiempo para hacerme pndja en el trabaj...,@estefaniasandoval9999,0,2024-08-19T19:46:46Z,6,Comentarios humorísticos o memes
1022,Ugy2h_vD89gf_8eeFph4AaABAg,t3MogAmkMUg,"Ya que son Otakus declarados, les recomiendo e...",@medinamartinezdavid8052,0,2024-08-19T19:33:41Z,2,Propuestas y recomendaciones
1023,Ugx7vBivUc-z-n8H76t4AaABAg,t3MogAmkMUg,"Pero dejen hablar al invitado, chavos banda!! :D",@hoffnerbass,1,2024-08-19T19:33:18Z,1,Quejas o sugerencias de mejora
1024,Ugzvu7xFsIUo8ibNXPJ4AaABAg,t3MogAmkMUg,El primer metal no sería el Na cuando el human...,@LuisAndresOtalvaroSanchez,0,2024-08-19T19:13:00Z,4,Correcciones o datos adicionales


In [7]:
texto = df_cleaned['textOriginal'][52]
texto

'40:57 Hay un anime donde su protagonista se llama Nike, está hermoso 🤩'

In [8]:
pattern = r'''(?x)                     # set flag to allow verbose regexps
              (?:[A-Z]\.)+             # abbreviations, e.g. U.S.A.
              | \d+:\d+                # numbers separated by a colon, e.g. 40:50
              | \$?\d+(?:\.\d+)?%?     # currency and percentages, e.g. $12.40, 82%
              | \w+(?:-\w+)*           # words with optional internal hyphens
              | \.\.\.                 # ellipsis

'''

# Consider special characters
pattern_sc = r'''(?x)                  # set flag to allow verbose regexps
              (?:[A-Z]\.)+             # abbreviations, e.g. U.S.A.
              | \d+:\d+                # numbers separated by a colon, e.g. 40:50
              | \$?\d+(?:\.\d+)?%?     # currency and percentages, e.g. $12.40, 82%
              | \w+(?:-\w+)*           # words with optional internal hyphens
              | \.\.\.                 # ellipsis
              | [][.,;"'?():-_`]       # these are separate tokens; includes ], [
'''

nltk.regexp_tokenize(texto + "?", pattern)

['40:57',
 'Hay',
 'un',
 'anime',
 'donde',
 'su',
 'protagonista',
 'se',
 'llama',
 'Nike',
 'está',
 'hermoso']

In [9]:
df_cleaned['Tokens_full'] = df_cleaned['textOriginal'].apply(lambda x: nltk.regexp_tokenize(x, pattern_sc))
df_cleaned

,id,videoId,textOriginal,authorDisplayName,likeCount,publishedAt,category_id,category_description,Tokens_full
0,Ugz0QmXyVLSM0nUK7kB4AaABAg,cQ9RLDhq6JY,"Me gusta su podcast, pero solo he escuchado lo...",@fabianbecerra2272,0,2024-07-15T12:41:43Z,1,Quejas o sugerencias de mejora,"[Me, gusta, su, podcast, ,, pero, solo, he, es..."
1,UgyJhRBhVcbyStZdxjR4AaABAg,cQ9RLDhq6JY,weeeeey,@SantiagoItzcoatl,0,2024-05-26T21:41:04Z,7,Comentarios generales,[weeeeey]
2,UgzT1q2am7kMEM5ckDp4AaABAg,cQ9RLDhq6JY,Les voy a enviar una foto de TARAPACA,@Sonyeke_1,0,2024-05-23T22:06:21Z,7,Comentarios generales,"[Les, voy, a, enviar, una, foto, de, TARAPACA]"
3,Ugyckucg38lqZFdZhaR4AaABAg,cQ9RLDhq6JY,Entonces Teca esta solterooooooooooooooo! ❤ ...,@alejandranavarro464,0,2024-05-05T09:24:57Z,6,Comentarios humorísticos o memes,"[Entonces, Teca, esta, solterooooooooooooooo, ..."
4,UgzhZHVt-tEbDpbCvtB4AaABAg,cQ9RLDhq6JY,"Me encantan sus podcast, los escucho en spotif...",@fiorellamartinez873,0,2024-04-21T21:09:08Z,5,Felicitaciones y agradecimientos,"[Me, encantan, sus, podcast, ,, los, escucho, ..."
...,...,...,...,...,...,...,...,...,...
1021,Ugyuq-tSoKFj7HEnAQ54AaABAg,t3MogAmkMUg,Justo a tiempo para hacerme pndja en el trabaj...,@estefaniasandoval9999,0,2024-08-19T19:46:46Z,6,Comentarios humorísticos o memes,"[Justo, a, tiempo, para, hacerme, pndja, en, e..."
1022,Ugy2h_vD89gf_8eeFph4AaABAg,t3MogAmkMUg,"Ya que son Otakus declarados, les recomiendo e...",@medinamartinezdavid8052,0,2024-08-19T19:33:41Z,2,Propuestas y recomendaciones,"[Ya, que, son, Otakus, declarados, ,, les, rec..."
1023,Ugx7vBivUc-z-n8H76t4AaABAg,t3MogAmkMUg,"Pero dejen hablar al invitado, chavos banda!! :D",@hoffnerbass,1,2024-08-19T19:33:18Z,1,Quejas o sugerencias de mejora,"[Pero, dejen, hablar, al, invitado, ,, chavos,..."
1024,Ugzvu7xFsIUo8ibNXPJ4AaABAg,t3MogAmkMUg,El primer metal no sería el Na cuando el human...,@LuisAndresOtalvaroSanchez,0,2024-08-19T19:13:00Z,4,Correcciones o datos adicionales,"[El, primer, metal, no, sería, el, Na, cuando,..."


In [10]:
df_cleaned['Tokens'] = df_cleaned['textOriginal'].apply(lambda x: nltk.regexp_tokenize(x, pattern))
df_cleaned

,id,videoId,textOriginal,authorDisplayName,likeCount,publishedAt,category_id,category_description,Tokens_full,Tokens
0,Ugz0QmXyVLSM0nUK7kB4AaABAg,cQ9RLDhq6JY,"Me gusta su podcast, pero solo he escuchado lo...",@fabianbecerra2272,0,2024-07-15T12:41:43Z,1,Quejas o sugerencias de mejora,"[Me, gusta, su, podcast, ,, pero, solo, he, es...","[Me, gusta, su, podcast, pero, solo, he, escuc..."
1,UgyJhRBhVcbyStZdxjR4AaABAg,cQ9RLDhq6JY,weeeeey,@SantiagoItzcoatl,0,2024-05-26T21:41:04Z,7,Comentarios generales,[weeeeey],[weeeeey]
2,UgzT1q2am7kMEM5ckDp4AaABAg,cQ9RLDhq6JY,Les voy a enviar una foto de TARAPACA,@Sonyeke_1,0,2024-05-23T22:06:21Z,7,Comentarios generales,"[Les, voy, a, enviar, una, foto, de, TARAPACA]","[Les, voy, a, enviar, una, foto, de, TARAPACA]"
3,Ugyckucg38lqZFdZhaR4AaABAg,cQ9RLDhq6JY,Entonces Teca esta solterooooooooooooooo! ❤ ...,@alejandranavarro464,0,2024-05-05T09:24:57Z,6,Comentarios humorísticos o memes,"[Entonces, Teca, esta, solterooooooooooooooo, ...","[Entonces, Teca, esta, solterooooooooooooooo, ..."
4,UgzhZHVt-tEbDpbCvtB4AaABAg,cQ9RLDhq6JY,"Me encantan sus podcast, los escucho en spotif...",@fiorellamartinez873,0,2024-04-21T21:09:08Z,5,Felicitaciones y agradecimientos,"[Me, encantan, sus, podcast, ,, los, escucho, ...","[Me, encantan, sus, podcast, los, escucho, en,..."
...,...,...,...,...,...,...,...,...,...,...
1021,Ugyuq-tSoKFj7HEnAQ54AaABAg,t3MogAmkMUg,Justo a tiempo para hacerme pndja en el trabaj...,@estefaniasandoval9999,0,2024-08-19T19:46:46Z,6,Comentarios humorísticos o memes,"[Justo, a, tiempo, para, hacerme, pndja, en, e...","[Justo, a, tiempo, para, hacerme, pndja, en, e..."
1022,Ugy2h_vD89gf_8eeFph4AaABAg,t3MogAmkMUg,"Ya que son Otakus declarados, les recomiendo e...",@medinamartinezdavid8052,0,2024-08-19T19:33:41Z,2,Propuestas y recomendaciones,"[Ya, que, son, Otakus, declarados, ,, les, rec...","[Ya, que, son, Otakus, declarados, les, recomi..."
1023,Ugx7vBivUc-z-n8H76t4AaABAg,t3MogAmkMUg,"Pero dejen hablar al invitado, chavos banda!! :D",@hoffnerbass,1,2024-08-19T19:33:18Z,1,Quejas o sugerencias de mejora,"[Pero, dejen, hablar, al, invitado, ,, chavos,...","[Pero, dejen, hablar, al, invitado, chavos, ba..."
1024,Ugzvu7xFsIUo8ibNXPJ4AaABAg,t3MogAmkMUg,El primer metal no sería el Na cuando el human...,@LuisAndresOtalvaroSanchez,0,2024-08-19T19:13:00Z,4,Correcciones o datos adicionales,"[El, primer, metal, no, sería, el, Na, cuando,...","[El, primer, metal, no, sería, el, Na, cuando,..."


Removing stopwords

In [11]:
stopwords_spanish = stopwords.words('spanish')

In [12]:
# Remove stopwords and punctuation
[ word for word in df_cleaned['Tokens'][52] if word not in stopwords_spanish and word not in string.punctuation]

['40:57', 'Hay', 'anime', 'protagonista', 'llama', 'Nike', 'hermoso']

In [13]:
df_cleaned['Tokens_without_stopwords'] = df_cleaned['Tokens'].apply(lambda x: [word for word in x if word not in stopwords_spanish and word not in string.punctuation])
df_cleaned

,id,videoId,textOriginal,authorDisplayName,likeCount,publishedAt,category_id,category_description,Tokens_full,Tokens,Tokens_without_stopwords
0,Ugz0QmXyVLSM0nUK7kB4AaABAg,cQ9RLDhq6JY,"Me gusta su podcast, pero solo he escuchado lo...",@fabianbecerra2272,0,2024-07-15T12:41:43Z,1,Quejas o sugerencias de mejora,"[Me, gusta, su, podcast, ,, pero, solo, he, es...","[Me, gusta, su, podcast, pero, solo, he, escuc...","[Me, gusta, podcast, solo, escuchado, Youtube,..."
1,UgyJhRBhVcbyStZdxjR4AaABAg,cQ9RLDhq6JY,weeeeey,@SantiagoItzcoatl,0,2024-05-26T21:41:04Z,7,Comentarios generales,[weeeeey],[weeeeey],[weeeeey]
2,UgzT1q2am7kMEM5ckDp4AaABAg,cQ9RLDhq6JY,Les voy a enviar una foto de TARAPACA,@Sonyeke_1,0,2024-05-23T22:06:21Z,7,Comentarios generales,"[Les, voy, a, enviar, una, foto, de, TARAPACA]","[Les, voy, a, enviar, una, foto, de, TARAPACA]","[Les, voy, enviar, foto, TARAPACA]"
3,Ugyckucg38lqZFdZhaR4AaABAg,cQ9RLDhq6JY,Entonces Teca esta solterooooooooooooooo! ❤ ...,@alejandranavarro464,0,2024-05-05T09:24:57Z,6,Comentarios humorísticos o memes,"[Entonces, Teca, esta, solterooooooooooooooo, ...","[Entonces, Teca, esta, solterooooooooooooooo, ...","[Entonces, Teca, solterooooooooooooooo, woooooo]"
4,UgzhZHVt-tEbDpbCvtB4AaABAg,cQ9RLDhq6JY,"Me encantan sus podcast, los escucho en spotif...",@fiorellamartinez873,0,2024-04-21T21:09:08Z,5,Felicitaciones y agradecimientos,"[Me, encantan, sus, podcast, ,, los, escucho, ...","[Me, encantan, sus, podcast, los, escucho, en,...","[Me, encantan, podcast, escucho, spotify, viaj..."
...,...,...,...,...,...,...,...,...,...,...,...
1021,Ugyuq-tSoKFj7HEnAQ54AaABAg,t3MogAmkMUg,Justo a tiempo para hacerme pndja en el trabaj...,@estefaniasandoval9999,0,2024-08-19T19:46:46Z,6,Comentarios humorísticos o memes,"[Justo, a, tiempo, para, hacerme, pndja, en, e...","[Justo, a, tiempo, para, hacerme, pndja, en, e...","[Justo, tiempo, hacerme, pndja, trabajo, bien,..."
1022,Ugy2h_vD89gf_8eeFph4AaABAg,t3MogAmkMUg,"Ya que son Otakus declarados, les recomiendo e...",@medinamartinezdavid8052,0,2024-08-19T19:33:41Z,2,Propuestas y recomendaciones,"[Ya, que, son, Otakus, declarados, ,, les, rec...","[Ya, que, son, Otakus, declarados, les, recomi...","[Ya, Otakus, declarados, recomiendo, anime, Dr..."
1023,Ugx7vBivUc-z-n8H76t4AaABAg,t3MogAmkMUg,"Pero dejen hablar al invitado, chavos banda!! :D",@hoffnerbass,1,2024-08-19T19:33:18Z,1,Quejas o sugerencias de mejora,"[Pero, dejen, hablar, al, invitado, ,, chavos,...","[Pero, dejen, hablar, al, invitado, chavos, ba...","[Pero, dejen, hablar, invitado, chavos, banda, D]"
1024,Ugzvu7xFsIUo8ibNXPJ4AaABAg,t3MogAmkMUg,El primer metal no sería el Na cuando el human...,@LuisAndresOtalvaroSanchez,0,2024-08-19T19:13:00Z,4,Correcciones o datos adicionales,"[El, primer, metal, no, sería, el, Na, cuando,...","[El, primer, metal, no, sería, el, Na, cuando,...","[El, primer, metal, Na, humano, utilizó, sal]"


In [14]:
# Stemming
stem = SnowballStemmer('spanish')
df_cleaned['Tokens_without_stopwords_stemmed'] = df_cleaned['Tokens_without_stopwords'].apply(lambda x: [stem.stem(word) for word in x])
df_cleaned

,id,videoId,textOriginal,authorDisplayName,likeCount,publishedAt,category_id,category_description,Tokens_full,Tokens,Tokens_without_stopwords,Tokens_without_stopwords_stemmed
0,Ugz0QmXyVLSM0nUK7kB4AaABAg,cQ9RLDhq6JY,"Me gusta su podcast, pero solo he escuchado lo...",@fabianbecerra2272,0,2024-07-15T12:41:43Z,1,Quejas o sugerencias de mejora,"[Me, gusta, su, podcast, ,, pero, solo, he, es...","[Me, gusta, su, podcast, pero, solo, he, escuc...","[Me, gusta, podcast, solo, escuchado, Youtube,...","[me, gust, podcast, sol, escuch, youtub, favor..."
1,UgyJhRBhVcbyStZdxjR4AaABAg,cQ9RLDhq6JY,weeeeey,@SantiagoItzcoatl,0,2024-05-26T21:41:04Z,7,Comentarios generales,[weeeeey],[weeeeey],[weeeeey],[weeeeey]
2,UgzT1q2am7kMEM5ckDp4AaABAg,cQ9RLDhq6JY,Les voy a enviar una foto de TARAPACA,@Sonyeke_1,0,2024-05-23T22:06:21Z,7,Comentarios generales,"[Les, voy, a, enviar, una, foto, de, TARAPACA]","[Les, voy, a, enviar, una, foto, de, TARAPACA]","[Les, voy, enviar, foto, TARAPACA]","[les, voy, envi, fot, tarapac]"
3,Ugyckucg38lqZFdZhaR4AaABAg,cQ9RLDhq6JY,Entonces Teca esta solterooooooooooooooo! ❤ ...,@alejandranavarro464,0,2024-05-05T09:24:57Z,6,Comentarios humorísticos o memes,"[Entonces, Teca, esta, solterooooooooooooooo, ...","[Entonces, Teca, esta, solterooooooooooooooo, ...","[Entonces, Teca, solterooooooooooooooo, woooooo]","[entonc, tec, solteroooooooooooooo, wooooo]"
4,UgzhZHVt-tEbDpbCvtB4AaABAg,cQ9RLDhq6JY,"Me encantan sus podcast, los escucho en spotif...",@fiorellamartinez873,0,2024-04-21T21:09:08Z,5,Felicitaciones y agradecimientos,"[Me, encantan, sus, podcast, ,, los, escucho, ...","[Me, encantan, sus, podcast, los, escucho, en,...","[Me, encantan, podcast, escucho, spotify, viaj...","[me, encant, podcast, escuch, spotify, viaj, t..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1021,Ugyuq-tSoKFj7HEnAQ54AaABAg,t3MogAmkMUg,Justo a tiempo para hacerme pndja en el trabaj...,@estefaniasandoval9999,0,2024-08-19T19:46:46Z,6,Comentarios humorísticos o memes,"[Justo, a, tiempo, para, hacerme, pndja, en, e...","[Justo, a, tiempo, para, hacerme, pndja, en, e...","[Justo, tiempo, hacerme, pndja, trabajo, bien,...","[just, tiemp, hac, pndja, trabaj, bien, invit]"
1022,Ugy2h_vD89gf_8eeFph4AaABAg,t3MogAmkMUg,"Ya que son Otakus declarados, les recomiendo e...",@medinamartinezdavid8052,0,2024-08-19T19:33:41Z,2,Propuestas y recomendaciones,"[Ya, que, son, Otakus, declarados, ,, les, rec...","[Ya, que, son, Otakus, declarados, les, recomi...","[Ya, Otakus, declarados, recomiendo, anime, Dr...","[ya, otakus, declar, recom, anim, dr, ston, es..."
1023,Ugx7vBivUc-z-n8H76t4AaABAg,t3MogAmkMUg,"Pero dejen hablar al invitado, chavos banda!! :D",@hoffnerbass,1,2024-08-19T19:33:18Z,1,Quejas o sugerencias de mejora,"[Pero, dejen, hablar, al, invitado, ,, chavos,...","[Pero, dejen, hablar, al, invitado, chavos, ba...","[Pero, dejen, hablar, invitado, chavos, banda, D]","[per, dej, habl, invit, chav, band, d]"
1024,Ugzvu7xFsIUo8ibNXPJ4AaABAg,t3MogAmkMUg,El primer metal no sería el Na cuando el human...,@LuisAndresOtalvaroSanchez,0,2024-08-19T19:13:00Z,4,Correcciones o datos adicionales,"[El, primer, metal, no, sería, el, Na, cuando,...","[El, primer, metal, no, sería, el, Na, cuando,...","[El, primer, metal, Na, humano, utilizó, sal]","[el, prim, metal, na, human, utiliz, sal]"


In [15]:
# Lemmatization
# Load spanish model pipeline
nlp = stanza.Pipeline(lang="es", processors="tokenize,mwt,pos,lemma")

df_cleaned['Tokens_lemmatized'] = df_cleaned['textOriginal'].apply(
    lambda x: [word.lemma for sentence in nlp(x).sentences for word in sentence.words]
)

df_cleaned['Tokens_without_stopwords_lemmatized'] = df_cleaned['Tokens_without_stopwords'].apply(
    lambda tokens: [nlp(word).sentences[0].words[0].lemma for word in tokens]
)

df_cleaned

2024-11-23 05:29:04 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-11-23 05:29:04 INFO: Downloaded file to /root/stanza_resources/resources.json
2024-11-23 05:29:05 INFO: Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |

2024-11-23 05:29:05 INFO: Using device: cpu
2024-11-23 05:29:05 INFO: Loading: tokenize
2024-11-23 05:29:05 INFO: Loading: mwt
2024-11-23 05:29:05 INFO: Loading: pos
2024-11-23 05:29:06 INFO: Loading: lemma
2024-11-23 05:29:06 INFO: Done loading processors!


,id,videoId,textOriginal,authorDisplayName,likeCount,publishedAt,category_id,category_description,Tokens_full,Tokens,Tokens_without_stopwords,Tokens_without_stopwords_stemmed,Tokens_lemmatized,Tokens_without_stopwords_lemmatized
0,Ugz0QmXyVLSM0nUK7kB4AaABAg,cQ9RLDhq6JY,"Me gusta su podcast, pero solo he escuchado lo...",@fabianbecerra2272,0,2024-07-15T12:41:43Z,1,Quejas o sugerencias de mejora,"[Me, gusta, su, podcast, ,, pero, solo, he, es...","[Me, gusta, su, podcast, pero, solo, he, escuc...","[Me, gusta, podcast, solo, escuchado, Youtube,...","[me, gust, podcast, sol, escuch, youtub, favor...","[yo, gustar, su, podcast, ,, pero, sólo, haber...","[yo, gustar, podcast, sólo, escuchar, youtube,..."
1,UgyJhRBhVcbyStZdxjR4AaABAg,cQ9RLDhq6JY,weeeeey,@SantiagoItzcoatl,0,2024-05-26T21:41:04Z,7,Comentarios generales,[weeeeey],[weeeeey],[weeeeey],[weeeeey],[weeeeey],[weeeeey]
2,UgzT1q2am7kMEM5ckDp4AaABAg,cQ9RLDhq6JY,Les voy a enviar una foto de TARAPACA,@Sonyeke_1,0,2024-05-23T22:06:21Z,7,Comentarios generales,"[Les, voy, a, enviar, una, foto, de, TARAPACA]","[Les, voy, a, enviar, una, foto, de, TARAPACA]","[Les, voy, enviar, foto, TARAPACA]","[les, voy, envi, fot, tarapac]","[él, ir, a, enviar, uno, foto, de, tarapaca]","[él, ir, enviar, foto, tarapaca]"
3,Ugyckucg38lqZFdZhaR4AaABAg,cQ9RLDhq6JY,Entonces Teca esta solterooooooooooooooo! ❤ ...,@alejandranavarro464,0,2024-05-05T09:24:57Z,6,Comentarios humorísticos o memes,"[Entonces, Teca, esta, solterooooooooooooooo, ...","[Entonces, Teca, esta, solterooooooooooooooo, ...","[Entonces, Teca, solterooooooooooooooo, woooooo]","[entonc, tec, solteroooooooooooooo, wooooo]","[entonces, teca, estar, solterooooooooooooooo,...","[entonces, teca, solterooooooooooooooo, woooooo]"
4,UgzhZHVt-tEbDpbCvtB4AaABAg,cQ9RLDhq6JY,"Me encantan sus podcast, los escucho en spotif...",@fiorellamartinez873,0,2024-04-21T21:09:08Z,5,Felicitaciones y agradecimientos,"[Me, encantan, sus, podcast, ,, los, escucho, ...","[Me, encantan, sus, podcast, los, escucho, en,...","[Me, encantan, podcast, escucho, spotify, viaj...","[me, encant, podcast, escuch, spotify, viaj, t...","[yo, encantar, su, podcast, ,, él, escuchar, e...","[yo, encantar, podcast, escuchar, spotify, via..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,Ugyuq-tSoKFj7HEnAQ54AaABAg,t3MogAmkMUg,Justo a tiempo para hacerme pndja en el trabaj...,@estefaniasandoval9999,0,2024-08-19T19:46:46Z,6,Comentarios humorísticos o memes,"[Justo, a, tiempo, para, hacerme, pndja, en, e...","[Justo, a, tiempo, para, hacerme, pndja, en, e...","[Justo, tiempo, hacerme, pndja, trabajo, bien,...","[just, tiemp, hac, pndja, trabaj, bien, invit]","[justo, a, tiempo, para, hacer, yo, pndir, en,...","[justo, tiempo, hacer, pndja, trabajo, bien, i..."
1022,Ugy2h_vD89gf_8eeFph4AaABAg,t3MogAmkMUg,"Ya que son Otakus declarados, les recomiendo e...",@medinamartinezdavid8052,0,2024-08-19T19:33:41Z,2,Propuestas y recomendaciones,"[Ya, que, son, Otakus, declarados, ,, les, rec...","[Ya, que, son, Otakus, declarados, les, recomi...","[Ya, Otakus, declarados, recomiendo, anime, Dr...","[ya, otakus, declar, recom, anim, dr, ston, es...","[ya, que, ser, otakus, declarado, ,, él, recom...","[ya, otakus, declarado, recomer, anime, dr, st..."
1023,Ugx7vBivUc-z-n8H76t4AaABAg,t3MogAmkMUg,"Pero dejen hablar al invitado, chavos banda!! :D",@hoffnerbass,1,2024-08-19T19:33:18Z,1,Quejas o sugerencias de mejora,"[Pero, dejen, hablar, al, invitado, ,, chavos,...","[Pero, dejen, hablar, al, invitado, chavos, ba...","[Pero, dejen, hablar, invitado, chavos, banda, D]","[per, dej, habl, invit, chav, band, d]","[pero, dejar, hablar, a, el, invitado, ,, chav...","[pero, dejar, hablar, invitado, chavo, banda, D]"
1024,Ugzvu7xFsIUo8ibNXPJ4AaABAg,t3MogAmkMUg,El primer metal no sería el Na cuando el human...,@LuisAndresOtalvaroSanchez,0,2024-08-19T19:13:00Z,4,Correcciones o datos adicionales,"[El, primer, metal, no, sería, el, Na, cuando,...","[El, primer, metal, no, sería, el, Na, cuando,.

In [16]:
df_cleaned[['Tokens', 'Tokens_without_stopwords_lemmatized']]

,Tokens,Tokens_without_stopwords_lemmatized
0,"[Me, gusta, su, podcast, pero, solo, he, escuc...","[yo, gustar, podcast, sólo, escuchar, youtube,..."
1,[weeeeey],[weeeeey]
2,"[Les, voy, a, enviar, una, foto, de, TARAPACA]","[él, ir, enviar, foto, tarapaca]"
3,"[Entonces, Teca, esta, solterooooooooooooooo, ...","[entonces, teca, solterooooooooooooooo, woooooo]"
4,"[Me, encantan, sus, podcast, los, escucho, en,...","[yo, encantar, podcast, escuchar, spotify, via..."
...,...,...
1021,"[Justo, a, tiempo, para, hacerme, pndja, en, e...","[justo, tiempo, hacer, pndja, trabajo, bien, i..."
1022,"[Ya, que, son, Otakus, declarados, les, recomi...","[ya, otakus, declarado, recomer, anime, dr, st..."
1023,"[Pero, dejen, hablar, al, invitado, chavos, ba...","[pero, dejar, hablar, invitado, chavo, banda, D]"
1024,"[El, primer, metal, no, sería, el, Na, cuando,...","[el, primero, metal, na, humano, utilizar, sal]"


In [17]:
df_cleaned.to_csv(INTERIM_COMMENTS_CLEANED_DIR, encoding='utf-8', index=False)

# Create corpus tokens for each video

In [18]:
INTERIM_COMMENTS_CLEANED_BY_VIDEO_DIR = INTERIM_FILES_DIR.getsyspath("Comments_cleaned_by_video.csv")
df_clean = pd.read_csv(INTERIM_COMMENTS_CLEANED_DIR)
df_video_corpus =pd.DataFrame()

In [19]:
df_video_corpus_1 = df_clean.groupby("videoId")["Tokens_without_stopwords_lemmatized"].apply(
    lambda x: [token for sublist in x for token in ast.literal_eval(sublist)]
).reset_index(name="Tokens_without_stopwords_lemmatized_combined")

df_video_corpus_2 = df_clean.groupby("videoId")["Tokens"].apply(
    lambda x: [token for sublist in x for token in ast.literal_eval(sublist)]
).reset_index(name="Tokens_combined")

df_video_corpus = df_video_corpus_1.merge(df_video_corpus_2, on="videoId", how="inner")

df_video_corpus

,videoId,Tokens_without_stopwords_lemmatized_combined,Tokens_combined
0,0IDFrBktWIA,"[hasta, acá, huele, a, garnacha, yo, encantar,...","[HASTA, ACÁ, HUELE, A, GARNACHA, Me, encanta, ..."
1,0kY6Qt1V8Kc,"[yo, escucha, el, tres, ase, recomendado, yo, ...","[Me, fui, a, escuchas, Los, tres, ases, recome..."
2,5Uac-LKCY4Y,"[saber, capitulo, historia, videojuego, chsm, ...","[saquen, capitulo, sobre, la, historia, de, lo..."
3,7Jo3NR5lgd8,"[27:30, chile, declarar, guerra, Japón, 13, ab...","[27:30, Chile, le, declara, la, guerra, a, Jap..."
4,CPCN1Lqzc2U,"[Lopez, morir, abrazar, hijo, hijo, coronel, p...","[Lopez, no, murió, abrazando, a, su, hijo, su,..."
5,FOFeh_vfcD8,"[hacer, episodio, defenza, boqueron, guerra, c...","[Hagan, un, episodio, sobre, la, defenza, de, ..."
6,FXxcNjTZ4qo,"[wow, that, looks, amazing, jjaja, q, pusar, b...","[Wow, that, looks, amazing, Jjaja, no, se, q, ..."
7,Go2KlhN4cQk,"[estar, escuchar, episodio, cambio, paralelo, ...","[Estaba, escuchando, este, episodio, y, no, se..."
8,HxMXLWqe9HQ,"[imperio, japonés, pt4, plis, él, faltar, mara...","[Imperio, japonés, pt4, plis, Les, faltaron, l..."
9,J5dH-jKp-ac,"[yo, ir, ir, infierno, reír, uds, chistesito, ...","[Me, voy, a, ir, al, infierno, por, reírme, co..."


In [20]:
df_video_corpus_3 = df_clean.groupby("videoId")["Tokens"].apply(
    lambda x: [" ".join(ast.literal_eval(tokens)) for tokens in x]
).reset_index(name="List_of_Tokens_combined")

df_video_corpus_4 = df_clean.groupby("videoId")["Tokens_without_stopwords_lemmatized"].apply(
    lambda x: [" ".join(ast.literal_eval(tokens)) for tokens in x]
).reset_index(name="List_of_WSL_Tokens_combined")

df_video_corpus = df_video_corpus.merge(df_video_corpus_3, on="videoId", how="inner")
df_video_corpus = df_video_corpus.merge(df_video_corpus_4, on="videoId", how="inner")
df_video_corpus

,videoId,Tokens_without_stopwords_lemmatized_combined,Tokens_combined,List_of_Tokens_combined,List_of_WSL_Tokens_combined
0,0IDFrBktWIA,"[hasta, acá, huele, a, garnacha, yo, encantar,...","[HASTA, ACÁ, HUELE, A, GARNACHA, Me, encanta, ...","[HASTA ACÁ HUELE A GARNACHA, Me encanta que us...","[hasta acá huele a garnacha, yo encantar usar ..."
1,0kY6Qt1V8Kc,"[yo, escucha, el, tres, ase, recomendado, yo, ...","[Me, fui, a, escuchas, Los, tres, ases, recome...","[Me fui a escuchas Los tres ases recomendado, ...","[yo escucha el tres ase recomendado, yo encant..."
2,5Uac-LKCY4Y,"[saber, capitulo, historia, videojuego, chsm, ...","[saquen, capitulo, sobre, la, historia, de, lo...",[saquen capitulo sobre la historia de los vide...,[saber capitulo historia videojuego chsm queja...
3,7Jo3NR5lgd8,"[27:30, chile, declarar, guerra, Japón, 13, ab...","[27:30, Chile, le, declara, la, guerra, a, Jap...",[27:30 Chile le declara la guerra a Japón el 1...,[27:30 chile declarar guerra Japón 13 abril 19...
4,CPCN1Lqzc2U,"[Lopez, morir, abrazar, hijo, hijo, coronel, p...","[Lopez, no, murió, abrazando, a, su, hijo, su,...",[Lopez no murió abrazando a su hijo su hijo er...,[Lopez morir abrazar hijo hijo coronel paragua...
5,FOFeh_vfcD8,"[hacer, episodio, defenza, boqueron, guerra, c...","[Hagan, un, episodio, sobre, la, defenza, de, ...",[Hagan un episodio sobre la defenza de boquero...,[hacer episodio defenza boqueron guerra chaco ...
6,FXxcNjTZ4qo,"[wow, that, looks, amazing, jjaja, q, pusar, b...","[Wow, that, looks, amazing, Jjaja, no, se, q, ...",[Wow that looks amazing Jjaja no se q puse per...,[wow that looks amazing jjaja q pusar buen yo ...
7,Go2KlhN4cQk,"[estar, escuchar, episodio, cambio, paralelo, ...","[Estaba, escuchando, este, episodio, y, no, se...",[Estaba escuchando este episodio y no se como ...,[estar escuchar episodio cambio paralelo histo...
8,HxMXLWqe9HQ,"[imperio, japonés, pt4, plis, él, faltar, mara...","[Imperio, japonés, pt4, plis, Les, faltaron, l...","[Imperio japonés pt4 plis, Les faltaron las ma...","[imperio japonés pt4 plis, él faltar maravilla..."
9,J5dH-jKp-ac,"[yo, ir, ir, infierno, reír, uds, chistesito, ...","[Me, voy, a, ir, al, infierno, por, reírme, co...",[Me voy a ir al infierno por reírme con uds co...,[yo ir ir infierno reír uds chistesito Ana Fra...


In [21]:
df_video_corpus.to_csv(INTERIM_COMMENTS_CLEANED_BY_VIDEO_DIR, encoding='utf-8', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2820e488-6f1b-466d-af14-a66826f012e3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>